## Import the scene graph file

In [ ]:
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# import pandas as pd

# # Authenticate
# drive = None
# def authenticate():
#   global drive
  
#   auth.authenticate_user()
#   gauth = GoogleAuth()
#   gauth.credentials = GoogleCredentials.get_application_default()
#   drive = GoogleDrive(gauth)

# #Download files
# def downloadFiles(fileIds):
#   authenticate()
  
#   for fileId in fileIds:    
    
#     downloaded = drive.CreateFile({"id": fileId[1]})
#     downloaded.GetContentFile(fileId[0])

In [ ]:
# #Download file if not existing
# try:
#   _ = open("funsd_full_feature_sg_train.json", "r")
# except:
#   downloadFiles([["funsd_full_feature_sg_train.json", "1RCnDFoJG9YLhiN21YXlnikcmS_NsiD2J"]])

# try:
#   _ = open("funsd_full_feature_sg_test.json", "r")
# except:
#   downloadFiles([["funsd_full_feature_sg_test.json", "1-5qLiidgT9M9LzwbqqJRF_rIDPqvWWR_"]])

In [ ]:
# # downloading the scene graph file
# import json
# with open('funsd_full_feature_sg_train.json', 'r') as f:
#     train_list_dict=json.load(f)
# with open('funsd_full_feature_sg_test.json', 'r') as f:
#     eval_list_dict=json.load(f)

In [7]:
# downloading the scene graph file
import json
with open('../data/funsd_full_feature_sg_train_v4.json', 'r') as f:
    train_list_dict=json.load(f)
with open('../data/funsd_full_feature_sg_test_v4.json', 'r') as f:
    eval_list_dict=json.load(f)

In [8]:
print(eval_list_dict.keys())
print(eval_list_dict['82253058_3059']['objects']['1']['text_density'])
print(eval_list_dict['82253058_3059']['objects']['4']['relations'])
print(eval_list_dict['82253058_3059']['objects']['1'].keys())

dict_keys(['82837252', '85201976', '86263525', '82251504', '93106788', '82573104', '87528321', '85240939', '82562350', '82253245_3247', '87093315_87093318', '83443897', '87332450', '89856243', '83635935', '82250337_0338', '82200067_0069', '92380595', '86236474_6476', '82491256', '85540866', '86244113', '83823750', '83594639', '87528380', '86220490', '87086073', '87147607', '82252956_2958', '83641919_1921', '82253058_3059', '87125460', '83624198', '82504862', '86328049_8050', '82254765', '86079776_9777', '83553333_3334', '83573282', '87137840', '87428306', '86230203_0206', '87594142_87594144', '91814768_91814769', '86075409_5410', '83772145', '82253362_3364', '85629964', '82092117', '83996357'])
0.0012610340479192938
{'0': {'id': 80034, 'object': 8737}, '1': {'id': 80035, 'object': 8738}, '2': {'id': 80036, 'object': 8739}, '3': {'id': 80037, 'object': 8740}, '4': {'id': 80038, 'object': 8742}, '5': {'id': 80039, 'object': 8743}, '6': {'id': 80040, 'object': 8744}, '7': {'id': 80041, 'o

## Define some pre-processing function

### Set **k** nearest neighbour for distance base weighted graph
This is used to generate visual feature and text density based graph embedding, to generate a new key for object dictionary named 'near gap'.

In [9]:
# selected top k gap distance features
def top_k_nearest(gap_dict, k):
  new_gap_dict = {}
  list1= sorted(gap_dict.values())
  top_k_value = list1[:k]
  for obj in gap_dict:
    if gap_dict[obj] in top_k_value:
      new_gap_dict[obj] = gap_dict[obj]
  return new_gap_dict

In [10]:
# default k is 2
for img in train_list_dict:
  for obj in train_list_dict[img]['objects']:
    new_gap_dict = top_k_nearest(train_list_dict[img]['objects'][obj]['gap'],2)
    train_list_dict[img]['objects'][obj]['near_gap'] = new_gap_dict

In [11]:
for img in eval_list_dict:
  for obj in eval_list_dict[img]['objects']:
    new_gap_dict = top_k_nearest(eval_list_dict[img]['objects'][obj]['gap'],2)
    eval_list_dict[img]['objects'][obj]['near_gap'] = new_gap_dict

### Some Searching Tools

In [12]:
# according to id to search text
bbox_id_text = {}
for l in train_list_dict:
  for obj in train_list_dict[l]['objects']:
    id = train_list_dict[l]['objects'][obj]['id']
    bbox_id_text[id] = {}
    bbox_id_text[id]['text'] = train_list_dict[l]['objects'][obj]['text']
for l in eval_list_dict:
  for obj in eval_list_dict[l]['objects']:
    id = eval_list_dict[l]['objects'][obj]['id']
    bbox_id_text[id] = {}
    bbox_id_text[id]['text'] = eval_list_dict[l]['objects'][obj]['text']

In [13]:
# according to id to search object id in a document
def globalid_to_localid(id):
  for l in train_list_dict:
    for obj in train_list_dict[l]['objects']:
      if id == train_list_dict[l]['objects'][obj]['id']:
        return obj
  for l in eval_list_dict:
    for obj in eval_list_dict[l]['objects']:
      if id == eval_list_dict[l]['objects'][obj]['id']:
        return obj

In [14]:
# this dictionary used to transfer object id to text_density/text_number/char_density/char_number/visual_embedding/
id_density_dict = {}
for l in eval_list_dict:
  id_density_dict[l] = {}
  for obj in eval_list_dict[l]['objects']:
    id_density_dict[l][obj] = eval_list_dict[l]['objects'][obj]['text_number']
for l in train_list_dict:
  id_density_dict[l] = {}
  for obj in train_list_dict[l]['objects']:
    id_density_dict[l][obj] = train_list_dict[l]['objects'][obj]['text_number']

### Change Scalar to High Dimensional Vectors

In [15]:
import math
import torch
# one dimensional feature embedding
def positionalencoding1d(d_model, feature_list):
    """
    :param d_model: dimension of the model
    :param feature_list: length of positions
    :return: length*d_model position matrix
    """
    if d_model % 2 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with "
                         "odd dim (got dim={:d})".format(d_model))
    pe = torch.zeros(len(feature_list), d_model)
    feats = torch.tensor(feature_list).unsqueeze(1)
    div_term = torch.exp((torch.arange(0, d_model, 2, dtype=torch.float) *
                         -(math.log(10000.0) / d_model)))
    pe[:, 0::2] = torch.sin(feats.float() * div_term)
    pe[:, 1::2] = torch.cos(feats.float() * div_term)
    pe = pe.tolist()
    return pe


## Extracting Information from training and evaluation json file

### For Gap Distance weighted GCN model

In [16]:
train_list_dict['91391310']['objects']['0']['relations']

{'0': {'id': 363944, 'object': 5557},
 '1': {'id': 363945, 'object': 5558},
 '2': {'id': 363946, 'object': 5559},
 '3': {'id': 363947, 'object': 5560},
 '4': {'id': 363948, 'object': 5561},
 '5': {'id': 363949, 'object': 5562},
 '6': {'id': 363950, 'object': 5563},
 '7': {'id': 363951, 'object': 5564},
 '8': {'id': 363952, 'object': 5565},
 '9': {'id': 363953, 'object': 5566},
 '10': {'id': 363954, 'object': 5567},
 '11': {'id': 363955, 'object': 5568},
 '12': {'id': 363956, 'object': 5569},
 '13': {'id': 363957, 'object': 5570},
 '14': {'id': 363958, 'object': 5571},
 '15': {'id': 363959, 'object': 5572},
 '16': {'id': 363960, 'object': 5573},
 '17': {'id': 363961, 'object': 5574},
 '18': {'id': 363962, 'object': 5575},
 '19': {'id': 363963, 'object': 5576},
 '20': {'id': 363964, 'object': 5577},
 '21': {'id': 363965, 'object': 5578},
 '22': {'id': 363966, 'object': 5579},
 '23': {'id': 363967, 'object': 5580},
 '24': {'id': 363968, 'object': 5581},
 '25': {'id': 363969, 'object': 558

In [17]:
# Generate local graph based training dataset objects and relations list
new_train_list_dict = {}
for l in train_list_dict:
  tem_dic = {}
  tem_dic['objects'] = []
  tem_dic['relationships'] = []
  
  for obj in train_list_dict[l]['objects']:
    tem_dic['objects'].append(obj)
    tem_rel = [obj,obj,0]
    tem_dic['relationships'].append(tem_rel)
    for rel in train_list_dict[l]['objects'][obj]['near_gap']:
      tem_rel = [obj,rel]
      tem_rel.append(train_list_dict[l]['objects'][obj]['near_gap'][rel])
      tem_dic['relationships'].append(tem_rel)
  new_train_list_dict[l] = tem_dic

In [18]:
# Generate local graph based validation dataset objects and relations list
new_eval_list_dict = {}
for l in eval_list_dict:
  tem_dic = {}
  tem_dic['objects'] = []
  tem_dic['relationships'] = []

  for obj in eval_list_dict[l]['objects']:
    tem_dic['objects'].append(obj)
    tem_rel = [obj,obj,0]
    tem_dic['relationships'].append(tem_rel)
    for rel in eval_list_dict[l]['objects'][obj]['near_gap']:
      tem_rel = [obj,rel]
      tem_rel.append(eval_list_dict[l]['objects'][obj]['near_gap'][rel])
      tem_dic['relationships'].append(tem_rel)
  new_eval_list_dict[l] = tem_dic

In [19]:
#Using for searching real node size
num_obj_dict = {}
for l in new_train_list_dict:
    num_obj_dict[l] = len(new_train_list_dict[l]['objects'])
for l in new_eval_list_dict:
    num_obj_dict[l] = len(new_eval_list_dict[l]['objects'])

In [20]:
# useless code only rename
list_dict_train={}
for l in new_train_list_dict:
    list_dict_train[l] = new_train_list_dict[l]
list_dict_test = {}
for l in new_eval_list_dict:
    list_dict_test[l] = new_eval_list_dict[l]

## Training and Validation dataframe generation

In [21]:
#generate the training and labeling information for each image
#this information can be used to transfer object id to corresponding visual, text density or other kinds of features.
density_list_train = []
label_list_train = []
img_list_train = []
label_dict_train = {}
for l in train_list_dict:
  label_dict_train[l] = []
  for obj in train_list_dict[l]['objects']:
    density_list_train.append(obj)
    label_list_train.append(train_list_dict[l]['objects'][obj]['category'])
    img_list_train.append(l)
    label_dict_train[l].append(train_list_dict[l]['objects'][obj]['category'])
  for obj in range(num_obj_dict[l],181):
    density_list_train.append(str(obj))
    label_list_train.append(str(-1))
    img_list_train.append(l)
    label_dict_train[l].append(str(-1))

In [22]:
# generating evalutaion labeling informaiton dictionary
density_list_eval = []
label_list_eval = []
label_dict_eval = {}
img_list_eval = []
for l in eval_list_dict:
  label_dict_eval[l] = []
  for obj in eval_list_dict[l]['objects']:
    density_list_eval.append(obj)
    label_list_eval.append(eval_list_dict[l]['objects'][obj]['category'])
    img_list_eval.append(l)
    label_dict_eval[l].append(eval_list_dict[l]['objects'][obj]['category'])
  for obj in range(num_obj_dict[l],181):
    density_list_eval.append(str(obj))
    label_list_eval.append(str(-1))
    img_list_eval.append(l)
    label_dict_eval[l].append(str(-1))

In [23]:
from pandas import DataFrame
df_train = DataFrame(density_list_train,columns=['density'])
df_train['label'] = label_list_train
df_train['image'] = img_list_train
df_eval = DataFrame(density_list_eval,columns=['density'])
df_eval['label'] = label_list_eval
df_eval['image'] = img_list_eval

In [24]:
df_train.head(181)

,density,label,image
0,0,question,92091873
1,1,question,92091873
2,2,header,92091873
3,3,question,92091873
4,4,question,92091873
...,...,...,...
176,176,-1,92091873
177,177,-1,92091873
178,178,-1,92091873
179,179,-1,92091873


In [25]:
df_train['image'][1]

'92091873'

In [26]:
label_list_train = df_train['label'].fillna('other').tolist()
label_list_eval = df_eval['label'].fillna('other').tolist()

In [27]:
df_train_clean = df_train[df_train['label'].notnull()][['density','label','image']]
df_eval_clean = df_eval[df_eval['label'].notnull()][['density','label','image']] 

In [28]:
df_train_clean.head()

,density,label,image
0,0,question,92091873
1,1,question,92091873
2,2,header,92091873
3,3,question,92091873
4,4,question,92091873


In [29]:
print(list_dict_train)

{'92091873': {'objects': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70'], 'relationships': [['0', '0', 0], ['0', '26', 17], ['0', '42', 12], ['1', '1', 0], ['1', '38', 13], ['1', '44', 13], ['2', '2', 0], ['2', '11', 10], ['2', '47', 5], ['3', '3', 0], ['3', '48', 9], ['3', '49', 0], ['3', '69', 9], ['4', '4', 0], ['4', '13', 9], ['4', '50', 1], ['5', '5', 0], ['5', '13', 3], ['5', '49', 3], ['6', '6', 0], ['6', '5', 4], ['6', '7', 4], ['6', '19', 2], ['7', '7', 0], ['7', '8', 2], ['7', '19', 1], ['8', '8', 0], ['8', '7', 2], ['8', '33', 2], ['8', '49', 1], ['9', '9', 0], ['9', '46', 14], ['9', '54', 4], ['10', '10', 0], ['

In [30]:
label_list_train = df_train_clean['label'].to_list()
label_list_eval = df_eval_clean['label'].to_list()

In [31]:
id_list_train = df_train_clean['density'].to_list()
id_list_eval = df_eval_clean['density'].to_list()

In [32]:
img_list_train = df_train_clean['image'].to_list()
img_list_eval = df_eval_clean['image'].to_list()

In [33]:
obj_list_train = []
for i in range(len(id_list_train)):
  tem = []
  tem = [id_list_train[i],img_list_train[i]]
  obj_list_train.append(tem)
obj_list_eval = []
for i in range(len(id_list_eval)):
  tem = []
  tem = [id_list_eval[i],img_list_eval[i]]
  obj_list_eval.append(tem)

In [34]:
print(len(obj_list_train))

26969


## Convert label into one-hot

In [35]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
num_class = len(list(set(label_list_train)))
lEnc = LabelEncoder()
lEnc.fit(np.unique(list(set(label_list_train))))
labels_one_hot_train = {}
for f in label_dict_train:
  num_labels = lEnc.transform(label_dict_train[f])
  labels_one_hot_train[f] = []
  for l in num_labels:
    to_add = [0]*num_class
    to_add[l]=1
    labels_one_hot_train[f].append(to_add)

labels_one_hot_eval = {}
for f in label_dict_eval:
  lEnc = LabelEncoder()
  lEnc.fit(np.unique(label_dict_eval[f]))
  num_labels = lEnc.transform(label_dict_eval[f])
  labels_one_hot_eval[f] = []
  for l in num_labels:
    to_add = [0]*num_class
    to_add[l]=1
    labels_one_hot_eval[f].append(to_add)

In [36]:
print(len(labels_one_hot_train))

149


## Build Graph

Build empty node list of **each image** for training and validation data

In [37]:
node_lists_train = {}
for l in new_train_list_dict:
  node_lists_train[l] = []
  for obj in new_train_list_dict[l]['objects']:
    node_lists_train[l].append(obj)
node_lists_eval = {}
for l in new_eval_list_dict:
  node_lists_eval[l] = []
  for obj in new_eval_list_dict[l]['objects']:
    node_lists_eval[l].append(obj)


In [38]:
node_lists_train

{'92091873': ['0',
  '1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44',
  '45',
  '46',
  '47',
  '48',
  '49',
  '50',
  '51',
  '52',
  '53',
  '54',
  '55',
  '56',
  '57',
  '58',
  '59',
  '60',
  '61',
  '62',
  '63',
  '64',
  '65',
  '66',
  '67',
  '68',
  '69',
  '70'],
 '91939637': ['0',
  '1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40'],
 '87533049': ['0',
  '1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',


Build empty local graph for each image. Each object key contains two empty list to store gap and related other objects

In [39]:
# build local graph
object_graph_dict_train = {}
for l in new_train_list_dict:
  object_graph_dict_train[l] = {}
  for obj in new_train_list_dict[l]['objects']:
    object_graph_dict_train[l][obj] = {}
    object_graph_dict_train[l][obj]['gap'] = []
    object_graph_dict_train[l][obj]['obj2'] = []

object_graph_dict_eval = {}
for l in new_eval_list_dict:
  object_graph_dict_eval[l] = {}
  for obj in new_eval_list_dict[l]['objects']:
    object_graph_dict_eval[l][obj] = {}
    object_graph_dict_eval[l][obj]['gap'] = []
    object_graph_dict_eval[l][obj]['obj2'] = []

In [40]:
object_graph_dict_train

{'92091873': {'0': {'gap': [], 'obj2': []},
  '1': {'gap': [], 'obj2': []},
  '2': {'gap': [], 'obj2': []},
  '3': {'gap': [], 'obj2': []},
  '4': {'gap': [], 'obj2': []},
  '5': {'gap': [], 'obj2': []},
  '6': {'gap': [], 'obj2': []},
  '7': {'gap': [], 'obj2': []},
  '8': {'gap': [], 'obj2': []},
  '9': {'gap': [], 'obj2': []},
  '10': {'gap': [], 'obj2': []},
  '11': {'gap': [], 'obj2': []},
  '12': {'gap': [], 'obj2': []},
  '13': {'gap': [], 'obj2': []},
  '14': {'gap': [], 'obj2': []},
  '15': {'gap': [], 'obj2': []},
  '16': {'gap': [], 'obj2': []},
  '17': {'gap': [], 'obj2': []},
  '18': {'gap': [], 'obj2': []},
  '19': {'gap': [], 'obj2': []},
  '20': {'gap': [], 'obj2': []},
  '21': {'gap': [], 'obj2': []},
  '22': {'gap': [], 'obj2': []},
  '23': {'gap': [], 'obj2': []},
  '24': {'gap': [], 'obj2': []},
  '25': {'gap': [], 'obj2': []},
  '26': {'gap': [], 'obj2': []},
  '27': {'gap': [], 'obj2': []},
  '28': {'gap': [], 'obj2': []},
  '29': {'gap': [], 'obj2': []},
  '30': 

Fill up the empty graph based on scene graph json file

In [41]:
#fill the empty graph
for sg in list_dict_train:
    obj_list = list_dict_train[sg]['objects']
    for rel_item in list_dict_train[sg]['relationships']:
        obj1 = str(rel_item[0])
        obj2 = str(rel_item[1])
        object_graph_dict_train[sg][rel_item[0]]['obj2'].append(obj2)
        object_graph_dict_train[sg][rel_item[0]]['gap'].append(rel_item[2])

#fill the empty graph
for sg in list_dict_test:
    obj_list = list_dict_test[sg]['objects']
    for rel_item in list_dict_test[sg]['relationships']:
        obj1 = str(rel_item[0])
        obj2 = str(rel_item[1])
        object_graph_dict_eval[sg][rel_item[0]]['obj2'].append(obj2)
        object_graph_dict_eval[sg][rel_item[0]]['gap'].append(rel_item[2])

In [42]:
print(object_graph_dict_train['0000971160'])

{'0': {'gap': [0, 26, 42], 'obj2': ['0', '9', '17']}, '1': {'gap': [0, 2, 1], 'obj2': ['1', '5', '15']}, '2': {'gap': [0, 24, 21], 'obj2': ['2', '15', '16']}, '3': {'gap': [0, 7, 25], 'obj2': ['3', '8', '12']}, '4': {'gap': [0, 20, 23], 'obj2': ['4', '13', '16']}, '5': {'gap': [0, 0, 0], 'obj2': ['5', '20', '22']}, '6': {'gap': [0, 10, 25], 'obj2': ['6', '11', '21']}, '7': {'gap': [0, 27, 27], 'obj2': ['7', '3', '21']}, '8': {'gap': [0, 7, 34], 'obj2': ['8', '3', '13']}, '9': {'gap': [0, 26, 6], 'obj2': ['9', '0', '17']}, '10': {'gap': [0, 7, 11], 'obj2': ['10', '5', '11']}, '11': {'gap': [0, 10, 4], 'obj2': ['11', '6', '20']}, '12': {'gap': [0, 25, 30], 'obj2': ['12', '3', '4']}, '13': {'gap': [0, 20, 22], 'obj2': ['13', '4', '19']}, '14': {'gap': [0, 11, 18], 'obj2': ['14', '1', '15']}, '15': {'gap': [0, 1, 14], 'obj2': ['15', '1', '21']}, '16': {'gap': [0, 19, 19, 15], 'obj2': ['16', '1', '14', '15']}, '17': {'gap': [0, 6, 13], 'obj2': ['17', '9', '18']}, '18': {'gap': [0, 36, 13], 

In [43]:

print(object_graph_dict_train['92091873'])


{'0': {'gap': [0, 17, 12], 'obj2': ['0', '26', '42']}, '1': {'gap': [0, 13, 13], 'obj2': ['1', '38', '44']}, '2': {'gap': [0, 10, 5], 'obj2': ['2', '11', '47']}, '3': {'gap': [0, 9, 0, 9], 'obj2': ['3', '48', '49', '69']}, '4': {'gap': [0, 9, 1], 'obj2': ['4', '13', '50']}, '5': {'gap': [0, 3, 3], 'obj2': ['5', '13', '49']}, '6': {'gap': [0, 4, 4, 2], 'obj2': ['6', '5', '7', '19']}, '7': {'gap': [0, 2, 1], 'obj2': ['7', '8', '19']}, '8': {'gap': [0, 2, 2, 1], 'obj2': ['8', '7', '33', '49']}, '9': {'gap': [0, 14, 4], 'obj2': ['9', '46', '54']}, '10': {'gap': [0, 7, 0, 7], 'obj2': ['10', '51', '54', '59']}, '11': {'gap': [0, 1, 0], 'obj2': ['11', '40', '51']}, '12': {'gap': [0, 12, 5], 'obj2': ['12', '4', '13']}, '13': {'gap': [0, 2, 1, 2], 'obj2': ['13', '24', '26', '39']}, '14': {'gap': [0, 3, 5], 'obj2': ['14', '39', '57']}, '15': {'gap': [0, 4, 6], 'obj2': ['15', '16', '56']}, '16': {'gap': [0, 4, 1], 'obj2': ['16', '15', '17']}, '17': {'gap': [0, 1, 3], 'obj2': ['17', '16', '33']}, 

In [44]:
node_size_train = []
for f in object_graph_dict_train:
  l = object_graph_dict_train[f].keys()
  node_size_train.append(len(l))

node_size_eval = []
for f in object_graph_dict_eval:
  l = object_graph_dict_eval[f].keys()
  node_size_eval.append(len(l))

In [45]:
max(node_size_train), max(node_size_eval)

(181, 108)

In [46]:
max_len = 181
col_train = {}
row_train = {}
weight_train = {}
for f in object_graph_dict_train:
  col_train[f] = []
  row_train[f] = []
  weight_train[f] = []
  for obj in object_graph_dict_train[f]:
    obj_rel_list = object_graph_dict_train[f][obj]['obj2']
    for obj2 in object_graph_dict_train[f]:
      if obj2 in obj_rel_list:
        for i in range(len(obj_rel_list)):
          if obj2 == obj_rel_list[i]:
            weight_train[f].append(1/(object_graph_dict_train[f][obj]['gap'][i]+1000))
            row_train[f].append(obj)
            col_train[f].append(obj2)
      else:
        weight_train[f].append(0.0)
        row_train[f].append(obj)
        col_train[f].append(obj2)


In [47]:
len(weight_train.keys()), len(row_train.keys()), len(col_train.keys())

(149, 149, 149)

In [48]:
# weight_train['92091873'] 
len(row_train['92091873']), len(col_train['92091873'])

(5041, 5041)

In [49]:
max_len = 181
col_eval = {}
row_eval = {}
weight_eval = {}
for f in object_graph_dict_eval:
  col_eval[f] = []
  row_eval[f] = []
  weight_eval[f] = []
  for obj in object_graph_dict_eval[f]:
    obj_rel_list = object_graph_dict_eval[f][obj]['obj2']
    for obj2 in object_graph_dict_eval[f]:
      if obj2 in obj_rel_list:
        for i in range(len(obj_rel_list)):
          if obj2 == obj_rel_list[i]:
            weight_eval[f].append(1/(object_graph_dict_eval[f][obj]['gap'][i]+1000))
            row_eval[f].append(obj)
            col_eval[f].append(obj2)
      else:
        weight_eval[f].append(0.0)
        row_eval[f].append(obj)
        col_eval[f].append(obj2)


In [84]:
#PROBLEMS
# how to determine the weight to the node itself
# how to padding the node

In [50]:
import scipy.sparse as sp
node_size = 181
adj_train = {}
for f in weight_train:
  adj_train[f] = sp.csr_matrix((weight_train[f], (row_train[f], col_train[f])), shape=(node_size, node_size))

In [51]:
adj_train.keys()

dict_keys(['92091873', '91939637', '87533049', '01073843', '92586242', '0012529284', '71341634', '0001477983', '91315069_91315070', '0060207528', '91161344_91161347', '71366499', '00922237', '91355841', '93380187', '01150773_01150774', '0060165115', '91914407', '92081358_1359', '660978', '0011974919', '81749056_9057', '00093726', '0000999294', '81186212', '0060077689', '87672097', '11875011', '00851772_1780', '0011859695', '81619511_9513', '0001456787', '81310636', '0060080406', '0011973451', '91104867', '00836244', '0001476912', '12052385', '0060255888', '71206427', '0012199830', '0001463448', '0000990274', '716552', '00920222', '0012529295', '80728670', '93213298', '71108371', '0001209043', '88057519', '01191071_1072', '00836816', '91361993', '92298125', '00040534', '0011838621', '0060029036', '0000989556', '0060024314', '0001129658', '0071032790', '00838511_00838525', '87682908', '0060214859', '00070353', '93351929_93351931', '0001438955', '81619486_9488', '00920294', '0000971160', 

In [52]:
adj_train['91939637'][40]

<1x181 sparse matrix of type '<class 'numpy.float64'>'
	with 41 stored elements in Compressed Sparse Row format>

In [53]:
adj_train['92091873'][70]

<1x181 sparse matrix of type '<class 'numpy.float64'>'
	with 71 stored elements in Compressed Sparse Row format>

In [54]:
import scipy.sparse as sp
node_size = 181
adj_eval = {}
for f in weight_eval:
  adj_eval[f] = sp.csr_matrix((weight_eval[f], (row_eval[f], col_eval[f])), shape=(node_size, node_size))

### With padding weight and nodes

In [55]:
'''
#padding methods
#weight for padding is -1
#node id if for padding nodes is -1
max_len = 181
col = {}
row = {}
weight = {}
for f in object_graph_dict:
  col[f] = []
  row[f] = []
  weight[f] = []
  for obj in object_graph_dict[f]:
    obj_rel_list = object_graph_dict[f][obj]['obj2']
    for i in range(len(obj_rel_list)):
      try:
        weight[f].append(1/object_graph_dict[f][obj]['gap'][i])
      except:
        weight[f].append(0.0)
      row[f].append(obj)
      col[f].append(obj_rel_list[i])
    # padding exsiting nodes to virtual nodes
    for j in range(len(obj_rel_list),max_len):
      weight[f].append(-1)
      row[f].append(str(j))
      col[f].append(str(j))
  # padding nodes and relationships
  for k in range(len(object_graph_dict[f].keys()),max_len):
    for m in range(max_len):
      weight[f].append(-1)
      row[f].append(str(j))
      col[f].append(str(j))
'''

"\n#padding methods\n#weight for padding is -1\n#node id if for padding nodes is -1\nmax_len = 181\ncol = {}\nrow = {}\nweight = {}\nfor f in object_graph_dict:\n  col[f] = []\n  row[f] = []\n  weight[f] = []\n  for obj in object_graph_dict[f]:\n    obj_rel_list = object_graph_dict[f][obj]['obj2']\n    for i in range(len(obj_rel_list)):\n      try:\n        weight[f].append(1/object_graph_dict[f][obj]['gap'][i])\n      except:\n        weight[f].append(0.0)\n      row[f].append(obj)\n      col[f].append(obj_rel_list[i])\n    # padding exsiting nodes to virtual nodes\n    for j in range(len(obj_rel_list),max_len):\n      weight[f].append(-1)\n      row[f].append(str(j))\n      col[f].append(str(j))\n  # padding nodes and relationships\n  for k in range(len(object_graph_dict[f].keys()),max_len):\n    for m in range(max_len):\n      weight[f].append(-1)\n      row[f].append(str(j))\n      col[f].append(str(j))\n"

## Build Model

In [56]:
# from inits import *
import tensorflow.compat.v1 as tf

flags = tf.app.flags
FLAGS = flags.FLAGS

# global unique layer ID dictionary for layer name assignment
_LAYER_UIDS = {}


def get_layer_uid(layer_name=''):
    """Helper function, assigns unique layer IDs."""
    if layer_name not in _LAYER_UIDS:
        _LAYER_UIDS[layer_name] = 1
        return 1
    else:
        _LAYER_UIDS[layer_name] += 1
        return _LAYER_UIDS[layer_name]


def sparse_dropout(x, keep_prob, noise_shape):
    """Dropout for sparse tensors."""
    random_tensor = keep_prob
    random_tensor += tf.random_uniform(noise_shape)
    dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool)
    pre_out = tf.sparse_retain(x, dropout_mask)
    return pre_out * (1./keep_prob)


def dot(x, y, sparse=False):
    """Wrapper for tf.matmul (sparse vs dense)."""
    if sparse:
        res = tf.sparse_tensor_dense_matmul(x, y)
    else:
        res = tf.matmul(x, y)
    return res


class Layer(object):
    """Base layer class. Defines basic API for all layer objects.
    Implementation inspired by keras (http://keras.io).

    # Properties
        name: String, defines the variable scope of the layer.
        logging: Boolean, switches Tensorflow histogram logging on/off

    # Methods
        _call(inputs): Defines computation graph of layer
            (i.e. takes input, returns output)
        __call__(inputs): Wrapper for _call()
        _log_vars(): Log all variables
    """

    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            layer = self.__class__.__name__.lower()
            name = layer + '_' + str(get_layer_uid(layer))
        self.name = name
        self.vars = {}
        logging = kwargs.get('logging', False)
        self.logging = logging
        self.sparse_inputs = False

    def _call(self, inputs):
        return inputs

    def __call__(self, inputs):
        with tf.name_scope(self.name):
            if self.logging and not self.sparse_inputs:
                tf.summary.histogram(self.name + '/inputs', inputs)
            outputs = self._call(inputs)
            if self.logging:
                tf.summary.histogram(self.name + '/outputs', outputs)
            return outputs

    def _log_vars(self):
        for var in self.vars:
            tf.summary.histogram(self.name + '/vars/' + var, self.vars[var])



class GraphConvolution(Layer):
    """Graph convolution layer."""
    def __init__(self, input_dim, output_dim, placeholders, dropout=0.,
                 sparse_inputs=False, act=tf.nn.relu, bias=False,
                 featureless=False, **kwargs):
        super(GraphConvolution, self).__init__(**kwargs)

        if dropout:
            self.dropout = placeholders['dropout']
        else:
            self.dropout = 0.

        self.act = act
        self.support = placeholders['support']
        self.sparse_inputs = sparse_inputs
        self.featureless = featureless
        self.bias = bias

        # helper variable for sparse dropout
        self.num_features_nonzero = placeholders['num_features_nonzero']

        with tf.variable_scope(self.name + '_vars'):
            for i in range(len(self.support)):
                self.vars['weights_' + str(i)] = glorot([input_dim, output_dim],
                                                        name='weights_' + str(i))
            if self.bias:
                self.vars['bias'] = zeros([output_dim], name='bias')

        if self.logging:
            self._log_vars()

    def _call(self, inputs):
        x = inputs

        # dropout
        if self.sparse_inputs:
            x = sparse_dropout(x, 1-self.dropout, self.num_features_nonzero)
        else:
            x = tf.nn.dropout(x, 1-self.dropout)

        # convolve
        supports = list()
        for i in range(len(self.support)):
            if not self.featureless:
                pre_sup = dot(x, self.vars['weights_' + str(i)],
                              sparse=self.sparse_inputs)
            else:
                pre_sup = self.vars['weights_' + str(i)]            
            support = dot(self.support[i], pre_sup, sparse=True)
            supports.append(support)
        output = tf.add_n(supports)

        # bias
        if self.bias:
            output += self.vars['bias']
			
		
        self.embedding = output #output
        return self.act(output)


2023-05-18 18:37:25.576500: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [58]:
# from metrics import *
import tensorflow.compat.v1 as tf

flags = tf.app.flags
FLAGS = flags.FLAGS

def glorot(shape, name=None):
    """Glorot & Bengio (AISTATS 2010) init."""
    init_range = np.sqrt(6.0/(shape[0]+shape[1]))
    initial = tf.random_uniform(shape, minval=-init_range, maxval=init_range, dtype=tf.float32)
    return tf.Variable(initial, name=name)

class Model(object):
    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            name = self.__class__.__name__.lower()
        self.name = name

        logging = kwargs.get('logging', False)
        self.logging = logging

        self.vars = {}
        self.placeholders = {}

        self.layers = []
        self.activations = []

        self.inputs = None
        self.outputs = None

        self.loss = 0
        self.accuracy = 0
        self.optimizer = None
        self.opt_op = None

    def _build(self):
        raise NotImplementedError

    def build(self):
        """ Wrapper for _build() """
        with tf.variable_scope(self.name):
            self._build()

        # Build sequential layer model
        self.activations.append(self.inputs)
        for layer in self.layers:
            hidden = layer(self.activations[-1])
            self.activations.append(hidden)
        self.outputs = self.activations[-1]

        # Store model variables for easy access
        variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.name)
        self.vars = {var.name: var for var in variables}

        # Build metrics
        self._loss()
        self._accuracy()

        self.opt_op = self.optimizer.minimize(self.loss)

    def predict(self):
        pass

    def _loss(self):
        raise NotImplementedError

    def _accuracy(self):
        raise NotImplementedError

    def save(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = saver.save(sess, "tmp/%s.ckpt" % self.name)
        print("Model saved in file: %s" % save_path)

    def load(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = "tmp/%s.ckpt" % self.name
        saver.restore(sess, save_path)
        print("Model restored from file: %s" % save_path)

In [59]:
class GCN(Model):
    def __init__(self, placeholders, input_dim, **kwargs):
        super(GCN, self).__init__(**kwargs)

        self.inputs = placeholders['features']
        self.input_dim = input_dim
        # self.input_dim = self.inputs.get_shape().as_list()[1]  # To be supported in future Tensorflow versions
        self.output_dim = placeholders['labels'].get_shape().as_list()[1]
        self.placeholders = placeholders

        self.optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)

        self.build()

    def _loss(self):
        # Weight decay loss
        for var in self.layers[0].vars.values():
            self.loss += FLAGS.weight_decay * tf.nn.l2_loss(var)

        # Cross entropy error
        self.loss += masked_softmax_cross_entropy(self.outputs, self.placeholders['labels'],
                                                  self.placeholders['labels_mask'])

    def _accuracy(self):
        self.accuracy = masked_accuracy(self.outputs, self.placeholders['labels'],
                                        self.placeholders['labels_mask'])
        self.pred = tf.argmax(self.outputs, 1)
        self.labels = tf.argmax(self.placeholders['labels'], 1)

    def _build(self):

        self.layers.append(GraphConvolution(input_dim=self.input_dim,
                                            output_dim=FLAGS.hidden1,
                                            placeholders=self.placeholders,
                                            act=tf.nn.relu,
                                            dropout=True,
                                            featureless=False,
                                            sparse_inputs=True,
                                            logging=self.logging))
        
        self.layers.append(GraphConvolution(input_dim=FLAGS.hidden1,
                                            output_dim=self.output_dim,
                                            placeholders=self.placeholders,
                                            act=lambda x: x, #
                                            dropout=True,
                                            logging=self.logging))

    def predict(self):
        return tf.nn.softmax(self.outputs)

In [60]:
def masked_softmax_cross_entropy(preds, labels, mask):
    """Softmax cross-entropy loss with masking."""
    print(preds)
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=labels)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)


def masked_accuracy(preds, labels, mask):
    """Accuracy with masking."""
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(labels, 1))

    accuracy_all = tf.cast(correct_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)

## Model Setting

In [61]:
from __future__ import division
from __future__ import print_function

import time
import tensorflow.compat.v1 as tf

from sklearn import metrics
import random
import os
import sys


# # Set random seed
# seed = random.randint(1, 200)
# np.random.seed(seed)
# tf.set_random_seed(seed)


# Settings
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"

flags = tf.app.flags
FLAGS = flags.FLAGS
for name in list(flags.FLAGS):
      delattr(flags.FLAGS,name)
flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_string('dataset', 'sencegraph', 'Dataset string.')
# 'gcn', 'gcn_cheby', 'dense'
flags.DEFINE_string('model', 'gcn', 'Model string.')
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 100, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 768, 'Number of units in hidden layer 1.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 0,
                   'Weight for L2 loss on embedding matrix.')  # 5e-4
flags.DEFINE_integer('early_stopping', 20,
                     'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')

## Input Features setup

### Node Feature is visual embedding

In [62]:
# wrong code for global level 
# transfer from node id to features
import scipy.sparse as sp
features_train = {}
for l in node_lists_train:
  features_train[l] = []
  for node in node_lists_train[l]:
    feature = np.array([0.0]*2048)
    feature += id_density_dict[l][str(node)]
    features_train[l].append(feature)
  for i in range(len(node_lists_train[l]),181):
    feature = np.array([0.0]*2048)
    features_train[l].append(feature)
  features_train[l]=sp.csr_matrix(features_train[l])

In [63]:
import scipy.sparse as sp
features_eval = {}
for l in node_lists_eval:
  features_eval[l] = []
  for node in node_lists_eval[l]:
    feature = np.array([0.0]*2048)
    feature += id_density_dict[l][str(node)]
    features_eval[l].append(feature)
  for i in range(len(node_lists_eval[l]),181):
    feature = np.array([0.0]*2048)
    features_eval[l].append(feature)
  features_eval[l]=sp.csr_matrix(features_eval[l])

### Node embedding is Text Density Features

In [64]:

# wrong code for global level 
# transfer from node id to features
import scipy.sparse as sp
features_train = {}
for l in node_lists_train:
  features_train[l] = []
  for node in node_lists_train[l]:
    feature = np.array([0.0]*768)
    feature += np.array(positionalencoding1d(768,[id_density_dict[l][str(node)]])[0])
    features_train[l].append(feature)
  for i in range(len(node_lists_train[l]),181):
    feature = np.array([0.0]*768)
    features_train[l].append(feature)
  features_train[l]=sp.csr_matrix(features_train[l])


In [65]:

import scipy.sparse as sp
features_eval = {}
for l in node_lists_eval:
  features_eval[l] = []
  for node in node_lists_eval[l]:
    feature = np.array([0.0]*768)
    feature += np.array(positionalencoding1d(768,[id_density_dict[l][str(node)]])[0])
    features_eval[l].append(feature)
  for i in range(len(node_lists_eval[l]),181):
    feature = np.array([0.0]*768)
    features_eval[l].append(feature)
  features_eval[l]=sp.csr_matrix(features_eval[l])

In [66]:
features_train

{'92091873': <181x768 sparse matrix of type '<class 'numpy.float64'>'
 	with 54528 stored elements in Compressed Sparse Row format>,
 '91939637': <181x768 sparse matrix of type '<class 'numpy.float64'>'
 	with 31488 stored elements in Compressed Sparse Row format>,
 '87533049': <181x768 sparse matrix of type '<class 'numpy.float64'>'
 	with 33792 stored elements in Compressed Sparse Row format>,
 '01073843': <181x768 sparse matrix of type '<class 'numpy.float64'>'
 	with 79104 stored elements in Compressed Sparse Row format>,
 '92586242': <181x768 sparse matrix of type '<class 'numpy.float64'>'
 	with 25344 stored elements in Compressed Sparse Row format>,
 '0012529284': <181x768 sparse matrix of type '<class 'numpy.float64'>'
 	with 52608 stored elements in Compressed Sparse Row format>,
 '71341634': <181x768 sparse matrix of type '<class 'numpy.float64'>'
 	with 79488 stored elements in Compressed Sparse Row format>,
 '0001477983': <181x768 sparse matrix of type '<class 'numpy.float6

In [67]:
features_train['92091873'][70]

<1x768 sparse matrix of type '<class 'numpy.float64'>'
	with 768 stored elements in Compressed Sparse Row format>

In [68]:
print(len(features_train))

149


### Sparse to tuple

In [69]:
def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose()
        values = mx.data
        shape = mx.shape
        return coords, values, shape
    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)
    return sparse_mx


def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return sparse_to_tuple(features)
for l in features_train:
  features_train[l] = preprocess_features(features_train[l])
  
for l in features_eval:
  features_eval[l] = preprocess_features(features_eval[l])


/var/folders/rq/73_t64xn2zxf07vggynh53800000gq/T/ipykernel_32103/3828355148.py:21: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


In [70]:
print(adj_train['91391310'][0].shape)

(1, 181)


## input setting

In [71]:
#define input dictionary
# num_obj_dict was defined to record the number of object in each document page
#
y_train = {}
train_mask = {}
for f in new_train_list_dict:
  y_train[f] = np.array([[0]*num_class]*adj_train[f].shape[0])
  y_train[f] = np.array(labels_one_hot_train[f])
  train_mask[f] = [False]*adj_train[f].shape[0]
  train_mask[f][:num_obj_dict[f]] = [True]*num_obj_dict[f]
y_val = {}
val_mask = {}
for f in new_eval_list_dict:
  y_val[f] = np.array([[0]*num_class]*adj_eval[f].shape[0])
  y_val[f] = np.array(labels_one_hot_eval[f])
  val_mask[f] = [False]*adj_eval[f].shape[0]
  val_mask[f][:num_obj_dict[f]] = [True]*num_obj_dict[f]

In [72]:
def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

def preprocess_adj(adj):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation."""
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))
    return sparse_to_tuple(adj_normalized)
support_train = {}
for f in adj_train:
  support_train[f] = [preprocess_adj(adj_train[f])]
support_eval = {}
for f in adj_eval:
  support_eval[f] = [preprocess_adj(adj_eval[f])]

In [73]:
print(features_train['91391310'][2][1])
print(y_train['91391310'].shape)

768
(181, 5)


In [74]:
# just change the number of support gpu
num_supports = 1
tf.compat.v1.disable_eager_execution()
# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape = (5,768)),
    'labels': tf.placeholder(tf.float32, shape=(None, 5)),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0.5, shape=()),
    # helper variable for sparse dropout
    'num_features_nonzero': tf.placeholder(tf.int32)
}


In [102]:
# define input dimension which is same as node feature size
model = GCN(placeholders, input_dim=768, logging=True)

Tensor("graphconvolution_4/SparseTensorDenseMatMul/SparseTensorDenseMatMul:0", shape=(None, 5), dtype=float32)


2023-05-18 19:41:25.267555: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_7' with dtype int32
	 [[{{node Placeholder_7}}]]
2023-05-18 19:41:25.267644: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_7' with dtype int32
	 [[{{node Placeholder_7}}]]
2023-05-18 19:41:25.276538: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype int64 and shape [?]
	 [[{{node Placeholder}}]]
2023-05-18 19:41:2

In [103]:
# Initialize session
session_conf = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=session_conf)


# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    feed_dict_val = construct_feed_dict(
        features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy, model.pred, model.labels, model.layers[0].embedding, model.layers[1].embedding], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], outs_val[2], outs_val[3], outs_val[4], outs_val[5]


# Init variables
sess.run(tf.global_variables_initializer())

cost_val = []

In [104]:
def construct_feed_dict(features, support, labels, labels_mask, placeholders):
    """Construct feed dictionary."""
    feed_dict = dict()
    feed_dict.update({placeholders['labels']: labels})
    feed_dict.update({placeholders['labels_mask']: labels_mask})
    feed_dict.update({placeholders['features']: features})
    feed_dict.update({placeholders['support'][i]: support[i]
                      for i in range(len(support))})
    feed_dict.update({placeholders['num_features_nonzero']: features[1].shape})
    return feed_dict

## Train GCN model

In [105]:
# Train model
for epoch in range(FLAGS.epochs):
    
    # Construct feed dictionary
    if epoch == FLAGS.epochs-1:
        output_dic_train = {}
        output_dic_test = {}
    loss = 0
    output_dic_train = {}
    for f in features_train:

        feed_dict = construct_feed_dict(
            features_train[f], support_train[f], y_train[f], train_mask[f], placeholders)
        feed_dict.update({placeholders['dropout']: FLAGS.dropout})

    # Training step
        outs = sess.run([model.opt_op, model.loss, model.accuracy,
                         model.layers[0].embedding, model.layers[1].embedding], feed_dict=feed_dict)
        loss += outs[1]
        output_dic_train[str(f)] = outs[3]
    print(loss)
    
    # Validation
    for f in features_eval:
        cost, acc, pred, labels, emb1, emb2 = evaluate(
            features_eval[f], support_eval[f], y_val[f], val_mask[f], placeholders)
        if epoch == FLAGS.epochs-1:
          output_dic_test[f] = emb1
print("Optimization Finished!")

177.67383348941803
172.9209662079811
172.08528685569763
171.34274470806122
170.82903414964676
170.65702867507935
170.75275945663452
170.85299277305603
170.9249148964882
170.32778787612915
170.70869010686874
171.0591579079628
170.34698444604874
170.83456510305405
170.65361964702606
170.59425061941147
170.34200650453568
170.06705105304718
170.6876242160797
170.20799136161804
170.52209556102753
170.28535521030426
170.2141159772873
169.9955050945282
169.77245020866394
170.29079711437225
170.81748235225677
169.7020404934883
170.14504617452621
169.77002823352814
170.2600965499878
170.34141582250595
169.9289983510971
170.23061126470566
169.93360322713852
169.85801833868027
170.11875039339066
170.25077706575394
170.08303898572922
170.4208820462227
170.0584356188774
170.12921559810638
170.07561993598938
170.10166722536087
169.9169790148735
170.10409516096115
169.75676810741425
170.4676792025566
169.5222334265709
170.35615807771683
169.3891959786415
169.9359170794487
169.91688257455826
169.75786

In [106]:
    # Validation
    for f in features_eval:
        cost, acc, pred, labels, emb1, emb2 = evaluate(
            features_eval[f], support_eval[f], y_val[f], val_mask[f], placeholders)
        if epoch == FLAGS.epochs-1:
          output_dic_test[f] = emb1

In [107]:
output_dic_train['92091873'].shape

(181, 768)

In [109]:
output_dic_test['82837252'].shape

(181, 768)

In [83]:
output_dic_train_vis = output_dic_train
output_dic_test_vis = output_dic_test

In [84]:
output_dic_train_den = output_dic_train
output_dic_test_den = output_dic_test

In [93]:
tf.__version__

'2.12.0'

In [131]:
output_dic_train

{'92091873': array([[-0.15773568, -0.05833437, -0.05693499, ..., -0.09680356,
         -0.11430454, -0.10404674],
        [-0.11632342, -0.05330949, -0.05036625, ..., -0.0634416 ,
         -0.10483866, -0.08516078],
        [-0.10034236, -0.05430038, -0.05402358, ..., -0.09121083,
         -0.09045195, -0.09829833],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]], dtype=float32),
 '91939637': array([[-0.08171212, -0.05844989, -0.05316278, ..., -0.08448486,
         -0.07764722, -0.08976807],
        [-0.12081362, -0.05662858, -0.05491684, ..., -0.08038183,
         -0.08751141, -0.08315254],
        [-0.11842198, -0.05552205, -0.05388334, ..., -0.07410476,
         -0.09716573, -0.08336034],
        ...,
        [ 0.   

## Matching with Object Classification Dataframe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [143]:
import pandas as pd
df_train = pd.read_pickle('/content/drive/MyDrive/funsd/funsd_object_gcn_visual_density_near_bert_base_gcn_bert_base_train_parsing1_parsing2.pkl')
df_test = pd.read_pickle('/content/drive/MyDrive/funsd/funsd_object_gcn_visual_density_near_bert_base_gcn_bert_base_test_parsing1_parsing2.pkl')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/funsd/funsd_object_gcn_visual_density_near_bert_base_gcn_bert_base_train_parsing1_parsing2.pkl'

In [140]:
df_train.head()

,density,label,image
0,0,question,92091873
1,1,question,92091873
2,2,header,92091873
3,3,question,92091873
4,4,question,92091873


In [141]:
print(df_train['level2_parse_emb'][0])
print(df_train['level2_parse_emb'][2])

KeyError: 'level2_parse_emb'

In [144]:
print(train_list_dict['0000971160']['objects']['1'].keys())

dict_keys(['id', 'box', 'category', 'text', 'relations', 'gap', 'text_density', 'text_number', 'char_density', 'char_number', 'parsing_level1', 'parsing_level2', 'near_gap'])


In [145]:
for img in output_dic_train_den:
  for i in range(len(train_list_dict[img]['objects'].keys())):
    train_list_dict[img]['objects'][str(i)]['gcn_token_number'] = output_dic_train_den[img][i] 

In [146]:
for img in output_dic_test_den:
  for i in range(len(eval_list_dict[img]['objects'].keys())):
    eval_list_dict[img]['objects'][str(i)]['gcn_token_number'] = output_dic_test_den[img][i] 

In [147]:
gcn_den_list_train = []

In [148]:
for img in train_list_dict:
  for obj in train_list_dict[img]['objects']:
    c_obj = train_list_dict[img]['objects'][obj]
    gcn_den_list_train.append(c_obj['gcn_token_number'])

In [149]:
gcn_den_list_test = []

In [150]:
for img in eval_list_dict:
  for obj in eval_list_dict[img]['objects']:
    c_obj = eval_list_dict[img]['objects'][obj]
    gcn_den_list_test.append(c_obj['gcn_token_number'])

In [156]:
len(gcn_den_list_train), df_train.shape

(7411, (26969, 3))

In [154]:
#df_train = DataFrame (text_list_train,columns=['text'])
df_train['gcn_near_token_number'] = gcn_den_list_train

ValueError: Length of values (7411) does not match length of index (26969)

In [ ]:
#df_test = DataFrame (text_list_test,columns=['text'])
df_test['gcn_near_token_number'] = gcn_den_list_test

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train.to_pickle('/content/drive/MyDrive/funsd/funsd_object_gcn_visual_density_near_bert_base_gcn_bert_base_train_parsing1_parsing2.pkl')
df_test.to_pickle('/content/drive/MyDrive/funsd/funsd_object_gcn_visual_density_near_bert_base_gcn_bert_base_test_parsing1_parsing2.pkl')

In [168]:
import pickle

with open("training_dataset.pkl", "wb") as fp:
    pickle.dump(train_list_dict , fp) 
 
with open("testing_dataset.pkl", "wb") as fp:
    pickle.dump(eval_list_dict , fp)

In [157]:
train_list_dict.to_pickle('../funsd/funsd_object_gcn_visual_density_near_bert_base_gcn_bert_base_train_parsing1_parsing2.pkl')
eval_list_dict.to_pickle('../funsd/funsd_object_gcn_visual_density_near_bert_base_gcn_bert_base_test_parsing1_parsing2.pkl')

AttributeError: 'dict' object has no attribute 'to_pickle'

In [159]:
print(train_list_dict['0000971160']['objects']['1'].keys())

dict_keys(['id', 'box', 'category', 'text', 'relations', 'gap', 'text_density', 'text_number', 'char_density', 'char_number', 'parsing_level1', 'parsing_level2', 'near_gap', 'gcn_token_number'])


In [165]:
train_list_dict['0000971160']['objects']['1']['gcn_token_number'].shape

(768,)